In [1]:
using Pkg; Pkg.activate("../tambo/")
Pkg.add("Roots")
using Plots

using Random, LinearAlgebra, Roots

using Revise
push!(LOAD_PATH, String(@__DIR__)*"/../tambo/src")
using Tracks
using Geometry: TPoint, is_inside, Box
using PyCall; np=pyimport("numpy")

  Activating project at `~/research/TAMBO-MC/tambo`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/research/TAMBO-MC/tambo/Project.toml`
  No Changes to `~/research/TAMBO-MC/tambo/Manifest.toml`
  0 dependencies successfully precompiled in 1 seconds (174 already precompiled)
  1 dependency errored. To see a full report either run `import Pkg; Pkg.precompile()` or load the package


LoadError: ArgumentError: Package Geometry not found in current path:
- Run `import Pkg; Pkg.add("Geometry")` to install the Geometry package.


# Make a ~~fake~~ real valley

In [2]:
# function f(x, center, width, intercept = -125)
#     x = x-center
#     a = (3000-intercept)/(width/2)^2
#     a*x^2+intercept
# end

# function my_valley(x, y)
#     ledge = 1000+200*sin(y/200)
#     redge = 2500+300*cos(y/750)
    
#     width = redge-ledge
#     center = (ledge + redge)/2
    
#     if x < ledge || x > redge
#         height = 3000
#     else
#         height = maximum((0, f(x, center, width)))
#     end
    
#     height += maximum((0, ((y)^2/1000+2500)*(abs(x-center)/25)^-1))
# #     height += maximum((0, (-(y-3000)^2/1000+2500)*(abs(x-center)/25)^-1))
#     height = minimum((height, 3000))
# end

# my_valley(1200, 3423)

x = np.load("../resources/tambo_spline.npy", allow_pickle=true)
spl = x[1]

function my_valley(x,y)
    height = spl(x,y)
    height = height[1]    
end 

box = Box(maximum(spl.get_knots()[1]), maximum(spl.get_knots()[2]), 5000)

LoadError: UndefVarError: np not defined

# Make function to determine if point is in rock or air

In [3]:
function is_in_mountain(x, y, z)
    is_inside(x, y, z, my_valley)
end

function is_in_mountain(p::TPoint)
    is_in_mountain(p.x, p.y, p.z)
end    

function is_in_mountain(t::Track)
    p = t.ipoint
    is_in_mountain(p)
end

LoadError: UndefVarError: TPoint not defined

In [4]:
n_points = 10000
# RNG for reproducibility
seed = 925
twista = MersenneTwister(seed)

#=
Generate initial points for the tracks to start at
These should be insidee the box
and be uniformly distributed in y and z
And be towards the low-x end of the box
=#
xx = rand(twista, n_points).*box.c2[1]
yy = rand(twista, n_points).*box.c2[2]
zz = rand(twista, n_points).*box.c2[3]
ipointt = [TPoint(x...) for x in zip(xx, yy, zz)]
#=
Sanity chack to make sure that all points are in the box
This should be true by construction
If this fails, it could mean the point generation is bad
or, more likely, the is_inside function is meessed up
=#
for pt in ipointt
    if !is_inside(pt, box)
        println("Oh no ! The initial pointis outside the box")
    end
end
#=
Generate initial directions for the tracks
These should roughly be pointing horizontal and to the right
i.e. θ∼π/2 and ϕ∼0 
=#
θθ = acos.(rand(twista, n_points).-1)
ϕϕ = rand(twista, n_points).*(2*pi)

dirr = [Direction(x...) for x in zip(θθ, ϕϕ)]

# Make the tracks
trackk  = [Track(x..., box) for x in zip(ipointt, dirr)]
# Now the tracks all start at the edge of the box and come in
rtrackk = Tracks.reverse.(trackk);

LoadError: UndefVarError: box not defined

# Define structure for tracking the path of the particle for plotting

In [5]:
# An homage
mutable struct Trayectory
    track::Track
    dλ::Float64
    x::Vector{Float64}
    y::Vector{Float64}
    z::Vector{Float64}
    current_λ
    function Trayectory(track::Track, dλ::Float64)
        new(track, dλ, [], [], [], 0.0)
    end
end

function step!(t::Trayectory)
    t.current_λ += t.dλ
    p = t.track(t.current_λ)
    append!(t.x, p.x)
    append!(t.y, p.y)
    append!(t.z, p.z)
end

function split_for_plot(x, y, z)
    region_change = diff([is_in_mountain(x...) for x in zip(x, y, z)]).!=0
    split_i       = [i for i in 1:length(region_change) if region_change[i]==1]
    split_track   = []
    old_idx       = 1
    current_color = ifelse(is_in_mountain(x[1], y[1], z[1]), :red, :black)
    for new_idx in split_i
        push!(split_track, (x[old_idx:new_idx], y[old_idx:new_idx], z[old_idx:new_idx], current_color))
        old_idx = new_idx
        current_color = ifelse(current_color==:black, :red, :black)
    end
    push!(split_track, tuple(x[old_idx:end], y[old_idx:end], z[old_idx:end], current_color))
    split_track
end

function split_for_plot(t::Trayectory)
    split_for_plot(t.x, t.y, t.z)
end

split_for_plot (generic function with 2 methods)

# Plot it

In [6]:
tr = rtrackk[1]

res         = 50
gifify      = false
show_track  = true
spatial_res = 200 # meter
dλ          = spatial_res/tr.norm

xx = LinRange(box.c1[1], box.c2[1], res)
yy = LinRange(box.c1[2], box.c2[2], res)


kwargs = Dict(:xlim=>(minimum((box.c1[1], box.c2[1])), maximum((box.c1[1], box.c2[1]))),
          :ylim=>(minimum((box.c1[2], box.c2[2])), maximum((box.c1[2], box.c2[2]))),
          :zlim=>(minimum((box.c1[2], box.c2[3])), maximum((box.c1[3], box.c2[3]))),
          :st=>:surface,
          :alpha=>0.9,
          :c=>cgrad(palette([:skyblue3, :skyblue2, :navajowhite3, :navajowhite3, :goldenrod4, :goldenrod4, :olivedrab, :olivedrab, :green, :green, :green, :green])),
          :colorbar=>false,
          :legend=>false,
         )

l   = @layout [a{0.7h} ; c{0.5w} d]
plt = surface(xx, yy, my_valley, layout=l, axis=([], false); kwargs...)
# This should be done with an imshow-like thing.... I think
surface!(plt[2], xx, yy, my_valley; camera=(0,90), kwargs...)
surface!(plt[3], xx, yy, my_valley; camera=(0,0), kwargs...)

if show_track
    λλ  = LinRange(0, 1, 50)
    pts = [tr.ipoint+λ*tr.direction for λ in λλ]
    segments = split_for_plot([getfield.(pts, s) for s in [:x, :y, :z]]...)
    for segment in segments
        for idx in 1:3
            plot!(plt[idx], segment[1:3]..., color=segment[4], legend=false)
        end
    end
end

display(plt)
Plots.savefig(plt, "/Users/jlazar/research/TAMBO-MC/figures/fake_valley.pdf")

if gifify && ~show_track # These options don't play well together. Sorry
trayectory = Trayectory(tr, dλ)    
    @gif while trayectory.current_λ <=1
        step!(trayectory)
        segments = split_for_plot(trayectory)
        for segment in segments
            for idx in 1:3
                plot!(plt[idx], segment[1:3]..., color=segment[4], legend=false)
            end
        end
        plot!(plt[1], camera=(minimum((90, trayectory.current_λ*100)), minimum((90, trayectory.current_λ*100))))
    end every 1
end

LoadError: UndefVarError: rtrackk not defined

In [7]:
for t in rtrackk[1:100]
    oned_valley = Tracks.reduce_f(t, my_valley)
    root_func(λ) = oned_valley(λ)-t(λ).z
    zeros = find_zeros(root_func, 0, 1)
    if length(zeros)>0
        plt = plot(0:0.01:1, oned_valley.(0:0.01:1))
        plot!(plt, 0:0.01:1, getfield.(t.(0:0.01:1), :z), zlim=(0,5000))
        scatter!(plt, zeros, getfield.(t.(zeros), :z))
        display(plt)
    end
end


LoadError: UndefVarError: rtrackk not defined

In [8]:
sc = scatter(xx[1:10000], yy[1:1000])
display(sc)
sc = scatter(xx[1:10000], zz[1:1000])
display(sc)
sc = scatter(yy[1:10000], zz[1:1000])
display(sc)

sc = scatter(θθ[1:10000], ϕϕ[1:1000])
display(sc)


LoadError: UndefVarError: xx not defined

In [9]:
air_density = 1.0
rock_density = 2.6

function total_column_depth(t::Track, valley::Function)
    oned_valley = Tracks.reduce_f(t, my_valley)
    root_func(λ) = oned_valley(λ)-t(λ).z
    zeros = find_zeros(root_func, 0, 1)
    rgen = vcat([0], zeros, [1])
    ranges = [(x[2]-x[1], is_in_mountain(t((x[1]+x[2])/2))) for x in zip(rgen[1:end-1], rgen[2:end])]
    cd = 0
    for x in ranges
        width, in_mountain = x
        density = in_mountain ? rock_density : air_density
        cd += width*t.norm*density
    end
    cd
end

total_column_depth (generic function with 1 method)

In [10]:
@time total_column_depth.(rtrackk[1:10000], Ref(my_valley))

LoadError: UndefVarError: rtrackk not defined